In [1]:
import pandas as pd 
#from numba import jit


tables = ["ethereum.aave.ez_deposits", 
          "ethereum.aave.ez_liquidations", 
          "ethereum.aave.ez_repayments", 
          "ethereum.aave.ez_borrows", 
          "ethereum.aave.ez_withdraws"]

dfDeposits = pd.read_csv(tables[0].replace(".", "_") + ".csv")
dfliquidations = pd.read_csv(tables[1].replace(".", "_") + ".csv")
dfRepayments = pd.read_csv(tables[2].replace(".", "_") + ".csv")
dfBorrows = pd.read_csv(tables[3].replace(".", "_") + ".csv")
dfWithdrawals = pd.read_csv(tables[4].replace(".", "_") + ".csv")

# Need to clean up data that is pulled from DB 
dfDeposits['Time'] = dfDeposits['block_timestamp'].astype('datetime64[ns]')
dfliquidations['Time'] = dfliquidations['block_timestamp'].astype('datetime64[ns]')
dfRepayments['Time'] = dfRepayments['block_timestamp'].astype('datetime64[ns]')
dfBorrows['Time'] = dfBorrows['block_timestamp'].astype('datetime64[ns]')
dfWithdrawals['Time'] = dfWithdrawals['block_timestamp'].astype('datetime64[ns]')


/tmp/ipykernel_11258/2919981745.py:12: DtypeWarning: Columns (5,6,9,10,13,17,18,21,22,25,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dfliquidations = pd.read_csv(tables[1].replace(".", "_") + ".csv")
/tmp/ipykernel_11258/2919981745.py:13: DtypeWarning: Columns (5,6,9,10,13,17,18,21,22,25,26,28,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  dfRepayments = pd.read_csv(tables[2].replace(".", "_") + ".csv")
/tmp/ipykernel_11258/2919981745.py:14: DtypeWarning: Columns (5,6,9,10,13,17,18,21,22,25,26,28,30,33,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBorrows = pd.read_csv(tables[3].replace(".", "_") + ".csv")


In [2]:
assets = dfDeposits['symbol'].unique()

In [3]:
# Lazy Way To Get Prices 

priceDict = {}

for asset in assets : 
    
    priceDict[asset] = dfDeposits[dfDeposits.symbol == asset].iloc[-1]['token_price']

In [4]:
priceDict

{'WETH': 1220.29,
 'DAI': 1.0,
 'AAVE': 54.935067465,
 'SNX': 1.719411251,
 'REN': 0.07937848579,
 'TUSD': 0.9958280502,
 'UNI': 5.34,
 'USDT': 1.009888705,
 'MKR': 509.877294402,
 'LINK': 6.029770294,
 'YFI': 6060.700113284,
 'WBTC': 16873.478982301,
 'ZRX': 0.1873294768,
 'USDC': 0.99917,
 'ENJ': 0.3171978727,
 'KNC': 3.316653621,
 'sUSD': 1.00448422,
 'MANA': 0.3892533784,
 'BUSD': 1.0,
 'BAT': 0.2250422779,
 'CRV': 0.5233086018,
 'GUSD': 0.9999671358,
 'BAL': 6.820383014,
 'xSUSHI': 1.807785883,
 'renFIL': 4.225854664,
 'RAI': 2.801944153,
 'AMPL': 1.058012829,
 'USDP': 0.9943281569,
 'DPI': 71.526085936,
 'FRAX': 0.9895462853,
 'FEI': 1.003098374,
 'stETH': 1201.324286831,
 'ENS': 13.969576099,
 'UST': 0.07764685095,
 'CVX': 4.150519566,
 '1INCH': 0.388556059,
 'LUSD': 1.022145257}

In [5]:
# Operations to clean Data

#@jit
def find_first_deposit(row):
    return dfDeposits[dfDeposits.depositor_address == row['user']]['Time'].min()

#@jit
def find_first_borrow(row):
    return dfBorrows[dfBorrows.borrower_address == row['user']]['Time'].min()

#@jit
def find_last_withdraw(row):
    return dfWithdrawals[dfWithdrawals.depositor_address == row['user']]['Time'].max()

#@jit
def find_last_liquidation(row):
    return dfliquidations[dfliquidations.borrower == row['user']]['Time'].max()

#@jit
def find_total_deposits(row):
    return dfDeposits[(dfDeposits.depositor_address == row['user']) & (dfDeposits.symbol == asset)]['issued_tokens'].sum()

#@jit
def find_total_withdrawals(row):
    return dfWithdrawals[(dfWithdrawals.depositor_address == row['user']) & (dfWithdrawals.symbol == asset)]['withdrawn_tokens'].sum()

#@jit
def find_total_liquidations(row):
    return dfliquidations[(dfliquidations.borrower == row['user']) & (dfliquidations.collateral_token_symbol == asset)]['liquidated_amount'].sum()

#@jit
def find_total_borrows(row):
    return dfBorrows[(dfBorrows.borrower_address == row['user']) & (dfBorrows.symbol == asset)]['borrowed_tokens'].sum()

#@jit
def find_total_repayments(row):
    return dfBorrows[(dfRepayments.borrower == row['user']) & (dfBorrows.symbol == asset)]['repayed_tokens'].sum()

#@jit
def total_deposits_usd(row):
    return dfDeposits[dfDeposits.depositor_address == row['user']]['supplied_usd'].sum()




In [6]:
# Only want to look at users that have supplied more than min amount in USD terms 
minAmount = 1000000

users = dfDeposits[dfDeposits.supplied_usd > minAmount]['depositor_address'].unique()

df = pd.DataFrame({'user' : users})



In [ ]:

"""
NOTE APPEARS TO BE SOME MINOR ISSUE WITH DATA ~~~ 
https://etherscan.io/tx/0x0a33776dcc36eaf3e57a3fa6ac7bdb1489565ab1bfb53cefae5bc45f3eba8f55
This shows as deposit but not withdrawal for that user -> need to take into account transfers of aTokens 
Also need to account for fact A Token is elastic i.e. grows in number as interest accrues... 

""" 

#### CLEAN UP THE FUCKING TRASH 

df['FirstDeposit'] = df.apply(find_first_deposit, axis=1)
df['FirstBorrow'] = df.apply(find_first_deposit, axis=1)
df['LastWithdraw'] = df.apply(find_last_withdraw, axis=1)
df['LastLiquidation'] = df.apply(find_last_liquidation, axis=1)

#assetSimple = ['USDC']
colList = []

for asset in assets : 
    df['TotalDeposits' + asset] = df.apply(find_total_deposits, axis=1)
    df['TotalWithdrawals' + asset] = df.apply(find_total_withdrawals, axis=1)
    df['TotalLiquidations' + asset] = df.apply(find_total_liquidations, axis=1)
    df['TotalBorrows' + asset] = df.apply(find_total_borrows, axis=1)
    df['TotalRepayments' + asset] = df.apply(find_total_borrows, axis=1)
    df['Total Collateral' + asset] = df['TotalDeposits' + asset] - df['TotalWithdrawals' + asset] - df['TotalLiquidations' + asset]
    df['Total Collateral USD' + asset] = df['Total Collateral' + asset]*priceDict[asset]
    colList.append('Total Collateral USD' + asset)
    
df['Total Collateral USD'] = df[colList].sum(axis=1)

df['End Date'] = dfDeposits['Time'].max()

df['TotalUSDSupplied'] = df.apply(total_deposits_usd, axis=1)
df['TotalDaysSinceEntry'] = (df['End Date'] - df['FirstDeposit']).dt.days
df['DaysUntilExit'] = (df['LastWithdraw'] - df['FirstDeposit']).dt.days


# number of deposits 
# number of withdrawals 
# number of borrows 

# total usd deposit value 
# total usd borrow value 
    

In [ ]:

### assume they still have 2% of what was supplied + have withdrawn tough to perfectly calc this especially if user leaves dust 
df['Exit'] = (df['Total Collateral USD'] / df['TotalUSDSupplied'] < 0.05) & (df['LastWithdraw'] < dfDeposits['Time'].max())


In [ ]:
df.to_csv("aaveExample.csv")

In [ ]:
# get insights on users that have left & are new 
api_key = 'df73fafff72674634df8d995d3753c3c735dbb1b'
#insightUsers = newUsers + lostUsers

insightUsers = list(df['user'])

headers = {'AccessKey': api_key}
base_url = 'https://pro-openapi.debank.com/v1'
total_balance = {}

for user in insightUsers:
    url = f'{base_url}/user/all_complex_protocol_list?id={user}'
    #print(url)
    res = requests.get(url, headers=headers)
    data = res.json()
    total_balance[user] = data
    #print(json.dumps(data, indent=4))

with open('./data.json', 'w') as f:
    json.dump(total_balance, f)
    
    
n = len(insightUsers)

cleanData = []

for user in insightUsers : 
    user_data = total_balance[user]
    address = user
    items = len(user_data)
    for j in range(items) : 
        item = user_data[j]
        
        portfolioInfo = item['portfolio_item_list']
        pItems = len(portfolioInfo)
        value = 0
        for p in range(pItems) : 
            value += item['portfolio_item_list'][p]['stats']['net_usd_value']
            #asset = 
        appendItem = {
            "User" : address,
            "Name" : item['name'],
            "Chain" : item['chain'],
            "Value" : value
        }
        
        cleanData.append(appendItem)
        
        
userDf = pd.DataFrame(cleanData)
userDf.to_csv("AAVEusers.csv")
